In [ ]:
# ! pip install clickhouse_connect

In [1]:
import pandas as pd
import clickhouse_connect
from clickhouse_connect.driver.tools import insert_file
from glob import glob
from datetime import datetime, date
import os

import requests
from typing import Literal, List, Optional
import pendulum
from pydantic import BaseModel, ValidationError, field_validator, TypeAdapter, Field
from pydantic.networks import IPvAnyAddress
from typing_extensions import TypedDict
import json
import time
import numpy as np

In [3]:
# authorization at ClickHouse
HOST= os.getenv("HOST")
PORT= os.getenv("PORT")
USERNAME= os.getenv("USERNAME")
PASSWORD= os.getenv("PASSWORD")

client = clickhouse_connect.get_client(host=HOST, port=PORT, username=USERNAME, password=PASSWORD)


In [40]:
# AppMetrika key
key = os.getenv("key")

application_int_id = 2398688

start_date = pendulum.now('Europe/Moscow').subtract(days=1).to_date_string()
# заменить здесь days=1 на days=7 для загрузки за последние 7 дней не включая текущую дату
end_date = pendulum.now('Europe/Moscow').subtract(days=1).to_date_string()

# список всех событий со всеми доступными полями (на момент создания документа)
content = {'clicks':'application_id,click_datetime,click_id,click_ipv6,click_timestamp,click_url_parameters,click_user_agent,publisher_id,publisher_name,tracker_name,tracking_id,touch_type,is_bot,city,country_iso_code,device_manufacturer,device_model,device_type,google_aid,oaid,ios_ifa,ios_ifv,os_name,os_version,windows_aid',

           'installations':'application_id,installation_id,attributed_touch_type,click_datetime,click_id,click_ipv6,click_timestamp,click_url_parameters,click_user_agent,profile_id,publisher_id,publisher_name,tracker_name,tracking_id,install_datetime,install_ipv6,install_receive_datetime,install_receive_timestamp,install_timestamp,is_reattribution,is_reinstallation,match_type,appmetrica_device_id,city,connection_type,country_iso_code,device_locale,device_manufacturer,device_model,device_type,google_aid,oaid,ios_ifa,ios_ifv,mcc,mnc,operator_name,os_name,os_version,windows_aid,app_package_name,app_version_name',

           'postbacks':'application_id,attributed_touch_type,click_datetime,click_id,click_ipv6,click_timestamp,click_url_parameters,click_user_agent,publisher_id,publisher_name,tracker_name,tracking_id,install_datetime,install_ipv6,install_timestamp,match_type,appmetrica_device_id,device_locale,device_manufacturer,device_model,device_type,google_aid,oaid,ios_ifa,os_name,os_version,windows_aid,app_package_name,app_version_name,conversion_datetime,conversion_timestamp,event_name,attempt_datetime,attempt_timestamp,cost_model,notifying_status,postback_url,postback_url_parameters,response_body,response_code',

           'events':'event_datetime,event_json,event_name,event_receive_datetime,event_receive_timestamp,event_timestamp,session_id,installation_id,appmetrica_device_id,city,connection_type,country_iso_code,device_ipv6,device_locale,device_manufacturer,device_model,device_type,google_aid,ios_ifa,ios_ifv,mcc,mnc,operator_name,original_device_model,os_name,os_version,profile_id,windows_aid,app_build_number,app_package_name,app_version_name,application_id',

           'profiles':'profile_id,appmetrica_gender,appmetrica_birth_date,appmetrica_notifications_enabled,appmetrica_name,appmetrica_crashes,appmetrica_errors,appmetrica_first_session_date,appmetrica_last_start_date,appmetrica_push_opens,appmetrica_push_send_count,appmetrica_sdk_version,appmetrica_sessions,android_id,appmetrica_device_id,city,connection_type,country_iso_code,device_manufacturer,device_model,device_type,google_aid,ios_ifa,ios_ifv,mcc,mnc,operator_name,os_name,os_version,windows_aid,app_build_number,app_framework,app_package_name,app_version_name',

           'revenue_events':'revenue_quantity,revenue_price,revenue_currency,revenue_product_id,revenue_order_id,revenue_order_id_source,is_revenue_verified,event_datetime,event_name,event_receive_datetime,event_receive_timestamp,event_timestamp,session_id,installation_id,android_id,appmetrica_device_id,appmetrica_sdk_version,city,connection_type,country_iso_code,device_ipv6,device_locale,device_manufacturer,device_model,event_datetime,google_aid,ios_ifa,ios_ifv,mcc,mnc,operator_name,original_device_model,os_version,profile_id,windows_aid,app_build_number,app_package_name,app_version_name',

           'deeplinks':'deeplink_url_host,deeplink_url_parameters,deeplink_url_path,deeplink_url_scheme,event_datetime,event_receive_datetime,event_receive_timestamp,event_timestamp,is_reengagement,profile_id,publisher_id,publisher_name,session_id,tracker_name,tracking_id,android_id,appmetrica_device_id,appmetrica_sdk_version,city,connection_type,country_iso_code,device_ipv6,device_locale,device_manufacturer,device_model,device_type,google_aid,ios_ifa,ios_ifv,mcc,mnc,original_device_model,os_version,windows_aid,app_build_number,app_package_name,app_version_name',

           'crashes':'crash,crash_datetime,crash_group_id,crash_id,crash_name,crash_receive_datetime,crash_receive_timestamp,crash_timestamp,appmetrica_device_id,city,connection_type,country_iso_code,device_ipv6,device_locale,device_manufacturer,device_model,device_type,google_aid,ios_ifa,ios_ifv,mcc,mnc,operator_name,os_name,os_version,profile_id,windows_aid,app_package_name,app_version_name,application_id',

           'errors':'error,error_datetime,error_id,error_name,error_receive_datetime,error_receive_timestamp,error_timestamp,appmetrica_device_id,city,connection_type,country_iso_code,device_ipv6,device_locale,device_manufacturer,device_model,device_type,google_aid,ios_ifa,ios_ifv,mcc,mnc,operator_name,os_name,os_version,profile_id,windows_aid,app_package_name,app_version_name,application_id',

           'push_tokens':'token,token_datetime,token_receive_datetime,token_receive_timestamp,token_timestamp,appmetrica_device_id,city,connection_type,country_iso_code,device_ipv6,device_locale,device_manufacturer,device_model,device_type,google_aid,ios_ifa,ios_ifv,mcc,mnc,operator_name,os_name,os_version,profile_id,windows_aid,app_package_name,app_version_name,application_id',

           'sessions_starts':'session_id,session_start_datetime,session_start_receive_datetime,session_start_receive_timestamp,session_start_timestamp,appmetrica_device_id,city,connection_type,country_iso_code,device_ipv6,device_locale,device_manufacturer,device_model,device_type,google_aid,ios_ifa,ios_ifv,mcc,mnc,operator_name,original_device_model,os_name,os_version,profile_id,windows_aid,app_build_number,app_package_name,app_version_name,application_id'
           }

# Модели для валидации ответа Метрики
# для https://appmetrica.yandex.ru/docs/ru/mobile-api/logs/ref/installations
class UniqueInstallation(BaseModel):
      application_id: int
      installation_id: str
      attributed_touch_type: Literal['click', 'impression', 'unknown']
      click_datetime: datetime
      click_id: str
      click_ipv6: IPvAnyAddress
      click_timestamp: datetime
      click_url_parameters: str
      click_user_agent: str
      profile_id: str
      publisher_id: int
      publisher_name: str
      tracker_name: str
      tracking_id: int
      install_datetime: datetime
      install_ipv6: IPvAnyAddress
      install_receive_datetime: datetime
      install_receive_timestamp: int
      install_timestamp: int
      is_reattribution: bool
      is_reinstallation: bool
      match_type: Literal['fingerprint', 'referrer', 'identifier', '']
      appmetrica_device_id: int
      city: str
      connection_type: Literal['wifi', 'cell', 'unknown']
      country_iso_code: str
      device_locale: str
      device_manufacturer: str
      device_model: str
      device_type: Literal['phone', 'tablet', 'unknown']
      google_aid: str
      oaid: str
      ios_ifa: str
      ios_ifv: str
      mcc: int
      mnc: int
      operator_name: str
      os_name: Literal['ios', 'android', 'windows']
      os_version: str
      windows_aid: str
      app_package_name: str
      app_version_name: str


class TotalInstallations(BaseModel):
  data: List[UniqueInstallation]
    
    
class Items(BaseModel):
  item_id: int
  item_name: str
  price: float
  item_availability_status:Optional[Literal["in_stock"]]= None
  discount: Optional[int]= None
  quantity: Optional[int]= None
  currency: Optional[str]= None
  value: Optional[float]= None

  item_category: Optional[int]= None
  item_category2: Optional[int]= None
  item_category3: Optional[int]= None
  item_category4: Optional[int]= None


class CartItems(BaseModel):
  item_id: int
  item_name: str
  price: float
  item_availability_status: Literal['in_stock', 'self_delivery_only', 'to_order', 'not_available']
  discount: float
  quantity: int


class ViewCart(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  #user_id: int
  loyalty_card: bool
  loyalty_status: Optional[Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert', 'empty']] = None
  personal_promo: Optional[str]= ''
  coupon: Optional[str]= ''
  cart_discount: float
  items: Optional[dict[str, CartItems]] = ''


class AuthorizationAnon(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']


class LoginEvent(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  method: str
  previous_screen: str

  @field_validator('authorized')
  def valid_authorization(cls, value):
    if value:
      return value
    raise ValueError(f"Login event should return authorized = True, {value} instead")


class GeneralEvents(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int


class PromocodeEvents(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  promocode_name: str = Field(pattern=r'Промокод')


class ProductDescription(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  item_id: int
  item_name: str
  block_type: Literal['description', 'characteristics'] = Field(alias='type')


class ProductImage(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  item_id: int
  item_name: str
  block_type: Literal['photo_gallery_main','buyers_photo_gallery'] = Field(alias='type')


class ProductReview(BaseModel):  # open_reviews  product_reviews_list
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  item_id: int
  item_name: str
  block_type: Literal['reviews','questions'] = Field(alias='type')


class ProductUnit(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  item_id: int
  item_name: str
  unit: int


class AddToWishlist(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  items: dict[str, Items]
  item_brand: str
  promotion_id: int
  item_list_name: str


class AddToEstimate(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  screen_name: str
  items: dict[str, Items]
  item_brand: str
  promotion_id: int
  item_list_name: str
  estimate_id: int


class EstimateCardShare(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  estimate_id: int
  #item_id: Optional[int]= None


class EstimateCreate(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  estimate_id: int
  screen_name: str


class Notifications(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  screen_name: str


class Orders(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  transaction_id: str
  file_name: Optional[str] = None
  method: Optional[str] = None


class GeneralSearch(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  qty_item: int
  qty_category: str
  search_term: str
  redirect: str


class SearchAddProductShare(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  item_id: int
  item_name: str


class SearchProducts(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  item_id: int
  item_name: str
  click_position: str


class SearchQuery(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  click_position: str
  search_term: str


class SearchSortChoice(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  choice: str
  search_term: str


class SearchSuggest(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  suggestion_field: str
  search_term	: str
  suggestion_type: str


class SearchSuggestion(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  text: str


class OrderHistorySearch(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  search_term	: str


class AddPaymentInfo(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  currency: str
  value	: float
  items: dict[str, Items]


class PromoSwipe(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  block_name: str
  item_id: int
  item_name: str


class ViewItemList(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  items: dict[str, Items]
  item_list_name: str


class ViewPromotion(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  items: dict[str, Items]
  creative_slot: str
  promotion_id: int


class PaymentStartSuccess(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  transaction_id: int
  payment_type: str


class FavouriteSortChoice(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  choice: str
  item_category: int

class CatalogSortChoice(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  choice: str
  item_category: int

class PushClick(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  push_from: str = Field(alias='from')


class AddToComparison(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  screen_name: str
  items: dict[str, Items]


class CatalogUnit(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  unit: int
  item_category: int

class AddShippingInfo(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  currency: str
  value	: float
  items: dict[str, Items]
  shipping_tier: str


class DeliveryAddressCommit(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  adress_type: str = Field(alias='type')


class CategoriesTreeOpen(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  item_id:	int
  item_name: str
  item_category: int


class DeliveryDatePick(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  date:	datetime
  delivery_rate:	str
  time_interval: str


class FilterApply(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  choice: str
  item_category: int


class ProductInstock(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  item_id: int
  item_name: str


class ProductSeeAlso(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  item_id: int
  item_name: str
  item_category: str


class ServicesSave(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  garbage_removal: str
  rent:	str
  lifting:	str
  unloading:	str
  manipulator:	str
  return_obj:	str = Field(alias='return')
  sawing:	str
  coloring:	str
  mounting:	str
  service_free_lifting:str


class SignUp(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  previous_screen: str
  method: str


class SmsRequest(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  account_exists: bool


class VideoView(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  item_id: int
  item_name: str


class ViewDetailsPromo(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  promo_type: str = Field(alias='type')


class FormError(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  error_text: str
  screen_name: str
  error_code: str
  error_type: str
  error_field: str


class OrderRefundRequestCardRepeat(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  transaction_id: str


class SelectPromotion(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  creative_slot: str
  promotion_id: str


class ProductReviewAdd(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  item_id: int
  item_name: str
  review_type: str = Field(alias='type')


class ProductReviewSend(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  item_id: int
  item_name: str
  rating: int
  review_type: str = Field(alias='type')


class UnavailableItemsOpen(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  count: str


class CategoryToWishlist(BaseModel):
  authorized:	bool
  city: Literal['msk', 'spb', 'vbg', 'pzv', 'nvr', 'tvr', 'lug', 'gtn', 'kgs',
                'rf', 'cheb', 'yar', 'vld', 'pskv', 'klg', 'rzn']
  device_id: str
  loyalty_status: Literal['profi', 'brave_union', 'spec', 'ekspert',
                          'master', 'expert', 'super-ekspert']
  user_id: int
  item_category: str




# https://appmetrica.yandex.ru/docs/ru/mobile-api/logs/ref/events
class UniqueEvent(BaseModel):
      event_datetime:str

      event_name: Literal['add_payment_info', 'add_shipping_info', 'add_to_estimate',
                          'add_to_wishlist', 'authorization_anon', 'cart_clear',
                          'catalog_sort', 'catalog_sort_choice', 'categories_tree_open',
                          'city_choice', 'contacts_add_new', 'delivery_address_commit',
                          'delivery_date_pick', 'delivery_master_pick',
                          'estimate_card_share', 'filter_apply', 'login', 'main_promo_swipe',
                          'open_reviews', 'order_cancel', 'order_card_share',
                          'order_files_download', 'order_history_search',
                          'order_number_copy', 'product_description', 'product_image',
                          'product_instock', 'product_promo_swipe', 'product_reviews_list',
                          'product_see_also', 'product_share', 'product_unit', 'search',
                          'search_add', 'search_filter', 'search_products', 'search_query',
                          'search_sort', 'search_sort_choice', 'search_suggest',
                          'search_suggest_category', 'search_suggest_products',
                          'search_suggestion', 'search_voice', 'services_save', 'sign_up',
                          'sms_fill', 'sms_request', 'video_view', 'view_cart',
                          'view_details_promo', 'view_item_list', 'view_promotion',
                          'contacts_add', 'form_error', 'order_refund_request',
                          'payment_start', 'push_click', 'select_promotion',
                          'cart_loyalty_approve', 'estimate_create', 'notification_on',
                          'search_category', 'use_loyalty_bonuses', 'cart_change_city',
                          'delivery_height_restrictions', 'pickup_base_save',
                          'bonus_card_pincode', 'catalog_unit', 'order_support_request',
                          'payment_success', 'contacts_save', 'order_card_repeat',
                          'product_specials', 'profile_edit_change', 'notification_off',
                          'product_review_add', 'product_review_send',
                          'unavailable_items_open', 'order_routing', 'profile_swipe',
                          'cart_add_loyalty', 'estimate_delete_product', 'promocode_get',
                          'promocode_sms_input', 'promocode_sms_request', 'shops_call',
                          'company_add', 'estimate_card_download', 'add_to_comparison',
                          'estimate_card_copy', 'profile_bonus_add', 'company_add_new',
                          'cart_analogue', 'order_change_city', 'orders_filter_apply',
                          'category_to_wishlist', 'favourite_filter_apply',
                          'shops_filter_apply', 'shops_navigation', 'favourite_sort_choice',
                          'profile_bonus_edit_change', 'order_change_cart', 'promocode_remind',
                          'cart_add_coupon', 'video_view_progress', 'product_promo',
                          'estimate_delete_job']

      event_json: str   

      event_receive_datetime:datetime
      event_receive_timestamp: int
      event_timestamp: int
      session_id: int
      installation_id:str
      appmetrica_device_id: int
      city:str
      connection_type: Literal['wifi', 'cell', 'unknown']
      country_iso_code:str
      device_ipv6: IPvAnyAddress
      device_locale:str
      device_manufacturer:str
      device_model:str
      device_type: Literal['phone', 'tablet', 'unknown']
      google_aid:str
      ios_ifa:str
      ios_ifv:str
      mcc: int
      mnc: int
      operator_name:str
      original_device_model:str
      os_name:str
      os_version:str
      profile_id:str
      windows_aid:str
      app_build_number: int
      app_package_name:str
      app_version_name:str
      application_id: int

class TotalEvents(BaseModel):
  data: list[UniqueEvent]
    
    
# https://appmetrica.yandex.ru/docs/ru/mobile-api/logs/ref/events
class UniqueProfile(BaseModel):
      profile_id:str
      appmetrica_gender:str
      appmetrica_birth_date:str
      appmetrica_notifications_enabled: bool
      appmetrica_name:str
      appmetrica_crashes: int
      appmetrica_errors: int
      appmetrica_first_session_date:str
      appmetrica_last_start_date:str
      appmetrica_push_opens: int
      appmetrica_push_send_count: int
      appmetrica_sdk_version: int
      appmetrica_sessions: int
      android_id:str
      appmetrica_device_id: int
      city:str
      connection_type: Literal['wifi', 'cell', 'unknown']
      country_iso_code:str
      device_manufacturer:str
      device_model:str
      device_type: Literal['phone', 'tablet', 'unknown']
      google_aid:str
      ios_ifa:str
      ios_ifv:str
      mcc: int
      mnc: int
      operator_name:str
      os_name:str
      os_version:str
      windows_aid:str
      app_build_number: int
      app_framework: int
      app_package_name:str
      app_version_name: str

class TotalProfiles(BaseModel):
  data: list[UniqueProfile]
    
    

class UniqueDeeplink(BaseModel):
      deeplink_url_host:str
      deeplink_url_parameters:str
      deeplink_url_path:str
      deeplink_url_scheme:Literal['http', 'https']
      event_datetime:str
      event_receive_datetime:datetime
      event_receive_timestamp: int
      event_timestamp: int
      is_reengagement: bool
      profile_id:str
      publisher_id: int
      publisher_name:str
      session_id: int
      tracker_name:str
      tracking_id: int
      android_id:str
      appmetrica_device_id: int
      appmetrica_sdk_version: int
      city:str
      connection_type: Literal['wifi', 'cell', 'unknown']
      country_iso_code:str
      device_ipv6: IPvAnyAddress
      device_locale:str
      device_manufacturer:str
      device_model:str
      device_type: Literal['phone', 'tablet', 'unknown']
      google_aid:str
      ios_ifa:str
      ios_ifv:str
      mcc: int
      mnc: int
      original_device_model:str
      os_version:str
      windows_aid:str
      app_build_number: int
      app_package_name:str
      app_version_name: str

class TotalDeeplinks(BaseModel):
  data: list[UniqueDeeplink]
    
    
class UniqueCrashe(BaseModel):
      crash:str
      crash_datetime:datetime
      crash_group_id: int
      crash_id: int
      crash_name:str
      crash_receive_datetime: datetime
      crash_receive_timestamp: datetime
      crash_timestamp: datetime
      appmetrica_device_id: int
      city:str
      connection_type: Literal['wifi', 'cell', 'unknown']
      country_iso_code:str
      device_ipv6: IPvAnyAddress
      device_locale:str
      device_manufacturer:str
      device_model:str
      device_type: Literal['phone', 'tablet', 'unknown']
      google_aid:str
      ios_ifa:str
      ios_ifv:str
      mcc: int
      mnc: int
      operator_name:str
      os_name:Literal['ios', 'android', 'windows']
      os_version:str
      profile_id:str
      windows_aid:str
      app_package_name:str
      app_version_name:str
      application_id: int

class TotalCrashes(BaseModel):
  data: list[UniqueCrashe]
    
    
class UniqueSession(BaseModel):
      session_id: int
      session_start_datetime: datetime
      session_start_receive_datetime: datetime
      session_start_receive_timestamp: datetime
      session_start_timestamp: datetime
      appmetrica_device_id: int
      city:str
      connection_type:Literal['wifi', 'cell', 'unknown']
      country_iso_code:str
      device_ipv6: IPvAnyAddress
      device_locale:str
      device_manufacturer:str
      device_model:str
      device_type: Literal['phone', 'tablet', 'unknown']
      google_aid:str
      ios_ifa:str
      ios_ifv:str
      mcc: int
      mnc: int
      operator_name:str
      original_device_model:str
      os_name: Literal['ios', 'android', 'windows']
      os_version:str
      profile_id:str
      windows_aid:str
      app_build_number: int
      app_package_name:str
      app_version_name:str
      application_id: int

class TotalSessions(BaseModel):
  data: list[UniqueSession]
    
# запрос к Метрике
def unified_requests(app_id: int, key: str, metric: str, date_since: str, date_until: str):
  print(f'Унифицированный запрос {metric}')
  response = requests.get(
      url=f"https://api.appmetrica.yandex.ru/logs/v1/export/{metric}.json",
      headers= {'Authorization': f"OAuth {key}"},
      params={
          'application_id' : app_id,
          'date_since': date_since,
          'date_until': date_until,
          'fields': content[metric],
      }
                        )

  if response.status_code == 200:
    if response.json()['data'] == []:
      return {response.status_code :'Что-то пошло не так, Метрика отдала пустой файл'}

    else:

      return {response.status_code : response.json()}

  else:
    return {response.status_code : response.text}


# Рекурсия запроса до получения ответа и валидация
def get_logs_data(app_id: int, key: str, metric: str, date_since: str, date_until: str):
  data = unified_requests(app_id, key, metric, date_since, date_until)
  status = list(data.keys())# статус ответа метрики: 200 - данные, другое - ошибки

  config_dict = {"installations": TotalInstallations,
                 "profiles": TotalProfiles,
                 "deeplinks": TotalDeeplinks,
                 "crashes": TotalCrashes,
                 "sessions_starts": TotalSessions,
                 "events": TotalEvents}

  events_config_dict = {'add_payment_info': AddPaymentInfo,
                          'add_shipping_info': AddShippingInfo,
                          'add_to_estimate': AddToEstimate,
                          'add_to_wishlist': AddToWishlist,
                          'authorization_anon': AuthorizationAnon,
                          'cart_clear': GeneralEvents,
                          'catalog_sort': GeneralEvents,
                          'catalog_sort_choice': CatalogSortChoice,
                          'categories_tree_open': CategoriesTreeOpen,
                          'city_choice': GeneralEvents,
                          'contacts_add_new': GeneralEvents,
                          'delivery_address_commit': DeliveryAddressCommit,
                          'delivery_date_pick': DeliveryDatePick,
                          'delivery_master_pick': GeneralEvents,
                          'estimate_card_share': EstimateCardShare,
                          'filter_apply': FilterApply,
                          'login': LoginEvent,
                          'main_promo_swipe': PromoSwipe,
                          'open_reviews': ProductReview,
                          'order_cancel': Orders,
                          'order_card_share': Orders,
                          'order_files_download': Orders,
                          'order_history_search': OrderHistorySearch,
                          'order_number_copy': Orders,
                          'product_description': ProductDescription,
                          'product_image': ProductImage,
                          'product_instock': ProductInstock,
                          'product_promo_swipe': PromoSwipe,
                          'product_reviews_list': ProductReview,
                          'product_see_also': ProductSeeAlso,
                          'product_share': SearchAddProductShare,
                          'product_unit': ProductUnit,
                          'search': GeneralSearch,
                          'search_add': SearchAddProductShare,
                          'search_filter': GeneralEvents,
                          'search_products': SearchProducts,
                          'search_query': SearchQuery,
                          'search_sort': GeneralEvents,
                          'search_sort_choice': SearchSortChoice,
                          'search_suggest': SearchSuggest,
                          'search_suggest_category': SearchSuggest,
                          'search_suggest_products': SearchSuggest,
                          'search_suggestion': SearchSuggestion,
                          'search_voice': GeneralEvents,
                          'services_save': ServicesSave,
                          'sign_up': SignUp,
                          'sms_fill': GeneralEvents,
                          'sms_request': SmsRequest,
                          'video_view': VideoView,
                          'view_cart': ViewCart,
                          'view_details_promo': ViewDetailsPromo,
                          'view_item_list': ViewItemList,
                          'view_promotion': ViewPromotion,
                          'contacts_add': GeneralEvents,
                          'form_error': FormError,
                          'order_refund_request': OrderRefundRequestCardRepeat,
                          'payment_start': PaymentStartSuccess,
                          'push_click': PushClick,
                          'select_promotion': SelectPromotion,
                          'estimate_create': EstimateCreate,
                          'notification_on': Notifications,
                          'order_support_request': Orders,
                          'payment_success': PaymentStartSuccess,
                          'order_card_repeat': OrderRefundRequestCardRepeat,
                          'notification_off': Notifications,
                          'product_review_add': ProductReviewAdd,
                          'product_review_send': ProductReviewSend,
                          'unavailable_items_open': UnavailableItemsOpen,
                          'order_routing': Orders,
                          'cart_add_loyalty': GeneralEvents,
                          'add_to_comparison': AddToComparison,
                          'category_to_wishlist': CategoryToWishlist

    }

  def _validate_model(model, data):
    try:
      model.parse_obj(data)
      return  ['No errors']

    except ValidationError as e:
      return json.loads(e.json())


  def _validate_json_event(data):
      event_exeptions = []

      for ev in range(len(data['data'])):
        ev_data = json.loads(data['data'][ev]['event_json'])
        data['data'][ev]['event_json'] = ev_data

        if data['data'][ev]['event_name'] in list(events_config_dict.keys()):
          try:
            model = events_config_dict[data['data'][ev]['event_name']]
            model.parse_obj(data['data'][ev]['event_json'])

          except ValidationError as e:
            exept = json.loads(e.json())
            exept[0]['loc'] = [ev] + exept[0]['loc']
            event_exeptions.append(exept[0])
            continue
      return event_exeptions


  if status[0] == 200:
    ev_exeptions = 'No errors'

    exeptions = _validate_model(config_dict[metric], data[200])

    if metric=='events':
      ev_exeptions = _validate_json_event(data[200])

    if ev_exeptions != 'No errors':
      if exeptions != 'No errors':
        exeptions = exeptions + ev_exeptions
      else:
        exeptions = ev_exeptions

    return {"data":data[200]['data'], "ValidationError":exeptions}



  elif status[0] == 202:
    print('Данные подготавливаются на сервере')
    time.sleep(60)
    return get_logs_data(app_id, key, metric, date_since, date_until)

  else:
    return {"data":"no data","Error":data}


# словарь со всеми данными
all_data = dict()



# цикл для получения, валидации и загрузки в ClickHouse
for metric in ['sessions_starts', 'crashes', 'deeplinks','installations', 'events', 'profiles']:    # content.keys():  # сейчас стоит словарь с тем, что отдаёт метрика, но если позже подключат остальные показатели, то вместо словаря просто закомментированное content.keys
# принимаемые параметры: числовой id приложения, токен OAuth, метрика, начало и конец интервала дат в формате 'yyyy-mm-dd HH:mm:ss'
    data = get_logs_data(application_int_id, key, metric, f'{start_date} 00:00:00', f'{end_date} 23:59:59')

# Строка передаётся только в случае ошибок. Ошибка выводится на экран и записывается как значение
    if type(data) == str:
        print(metric, data)

    all_data[metric] = data
    
    if all_data[metric]['data'] != "no data":
        df = pd.DataFrame(all_data[metric]['data'])

        df = df.replace('', np.nan)

        df_columns = list(df.columns)

        for col in df_columns:
            if 'datetime' in col:
                df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')
            elif 'date' in col:
                df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
            elif 'timestamp' in col:
                df[col] = pd.to_datetime(df[col], unit='s')

        filename = f'{metric}_data.csv'
        df.to_csv(filename)

        insert_file(client, f'a_ostrovskaia_appmetrica_{metric}', filename,
                fmt="CSVWithNames",
                database='temporary' )
        
        print(f"Данные {metric} успешно загружены в ClickHouse")
    else:
        print(f"В данных {metric} ошибка, невозможно загрузить в ClickHouse")



Унифицированный запрос sessions_starts
Данные подготавливаются на сервере
Унифицированный запрос sessions_starts
Данные подготавливаются на сервере
Унифицированный запрос sessions_starts


C:\Users\THUNDEROBOT\AppData\Local\Temp\ipykernel_36120\1093424280.py:1112: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  model.parse_obj(data)
C:\Users\THUNDEROBOT\AppData\Local\Temp\ipykernel_36120\1093424280.py:1193: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df[col] = pd.to_datetime(df[col], unit='s')
C:\Users\THUNDEROBOT\AppData\Local\Temp\ipykernel_36120\1093424280.py:1193: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavi

Унифицированный запрос crashes
Данные подготавливаются на сервере
Унифицированный запрос crashes


C:\Users\THUNDEROBOT\AppData\Local\Temp\ipykernel_36120\1093424280.py:1112: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  model.parse_obj(data)
C:\Users\THUNDEROBOT\AppData\Local\Temp\ipykernel_36120\1093424280.py:1193: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df[col] = pd.to_datetime(df[col], unit='s')
C:\Users\THUNDEROBOT\AppData\Local\Temp\ipykernel_36120\1093424280.py:1193: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavi

Унифицированный запрос deeplinks
Данные подготавливаются на сервере
Унифицированный запрос deeplinks
Данные подготавливаются на сервере
Унифицированный запрос deeplinks


...

C:\Users\THUNDEROBOT\AppData\Local\Temp\ipykernel_36120\1093424280.py:1112: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  model.parse_obj(data)
C:\Users\THUNDEROBOT\AppData\Local\Temp\ipykernel_36120\1093424280.py:1193: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df[col] = pd.to_datetime(df[col], unit='s')
C:\Users\THUNDEROBOT\AppData\Local\Temp\ipykernel_36120\1093424280.py:1193: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavi

Унифицированный запрос events
Данные подготавливаются на сервере
Унифицированный запрос events


C:\Users\THUNDEROBOT\AppData\Local\Temp\ipykernel_36120\1093424280.py:1112: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  model.parse_obj(data)
C:\Users\THUNDEROBOT\AppData\Local\Temp\ipykernel_36120\1093424280.py:1129: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  model.parse_obj(data['data'][ev]['event_json'])
C:\Users\THUNDEROBOT\AppData\Local\Temp\ipykernel_36120\1093424280.py:1193: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or 

Унифицированный запрос profiles
Данные подготавливаются на сервере
Унифицированный запрос profiles
Данные подготавливаются на сервере
Унифицированный запрос profiles


C:\Users\THUNDEROBOT\AppData\Local\Temp\ipykernel_36120\1093424280.py:1112: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  model.parse_obj(data)
